In [1]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/item_feature_python_extract_Training_all.csv', encoding = 'utf8')

<ipython-input-4-c3c8f57a4438>:1: DtypeWarning: Columns (10,12,15,19,20,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/item_feature_python_extract_Training_all.csv', encoding = 'utf8')


In [5]:
# List of unique accessions
unique_accessions = ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']



# Dictionary to store the dataframes
dfs = {}

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the dataframe name
    path = '/content/drive/MyDrive/NAEP_Comp/'
    df_name = 'df_' + accession

    # Read the CSV file into a dataframe
    df = pd.read_csv(path + df_name + '.csv')

    # Add the dataframe to the dictionary
    dfs[accession] = df

<ipython-input-5-4e4dfdd30b27>:17: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + df_name + '.csv')


In [6]:
df = dfs['VH304954']

In [7]:
df['predict_from']

0        Mark needs to do next is to add 5 to the answe...
1                             he needs to take away 43 -48
2         What he needs to do next is subtract 5 from 100.
3                          he needs to subtract 5 from 100
4                            he needs to solve the problem
                               ...                        
19550                                 subtract 45 from 100
19551    Subtract 95 from 143-43 which the answer for 1...
19552                     he needs to subtract 100 from 48
19553                                    the answer is 405
19554                   know he need's to subtract  143-48
Name: predict_from, Length: 19555, dtype: object

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
rest_texts, test_texts, rest_labels, test_labels = train_test_split(df['predict_from'], df['assigned_score'], test_size=0.1, random_state=1)
train_texts, dev_texts, train_labels, dev_labels = train_test_split(rest_texts, rest_labels, test_size=0.1, random_state=1)


In [10]:
print("Train size:", len(train_texts))
print("Dev size:", len(dev_texts))
print("Test size:", len(test_texts))

Train size: 15839
Dev size: 1760
Test size: 1956


In [11]:
train_labels = train_labels-1
dev_labels = dev_labels-1
test_labels = test_labels-1

In [9]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
MODEL = "distilbert-base-uncased"

In [11]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from datasets import Dataset,load_dataset, load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, AdamW

# Create model and tokenizer
# Make sure the num_labels argument matches the question (it will usually be 2, for correct/incorrect)
# Some questions may require more than one model (for more than one written section)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [16]:
# Some questions may require more than one model (for more than one written section)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [13]:
df['labels'] = df['score_to_predict'] - 1

In [14]:
# Convert to dataset format
dataset = Dataset.from_pandas(df, preserve_index=False)
dataset = dataset.train_test_split(test_size=0.2, seed=11)
dataset

DatasetDict({
    train: Dataset({
        features: ['student_id', 'accession', 'score_to_predict', 'predict_from', 'year', 'srace10', 'dsex', 'accom2', 'iep', 'lep', 'rater_1', 'pta_rtr1', 'ptb_rtr1', 'ptc_rtr1', 'composite', 'score', 'assigned_score', 'ee_use', 'parsed_xml_v1', 'parsed_xml_v2', 'parsed_xml_v3', 'source1', 'source2', 'source3', 'source4', 'target1', 'target2', 'target3', 'target4', 'eliminations', 'selected', 'eliminated', 'selected1', 'selected2', 'selected3', 'selected4', 'eliminated1', 'eliminated2', 'eliminated3', 'eliminated4', 'selected1.1', 'selected2.1', 'eliminated1.1', 'eliminated2.1', 'partA_response_val', 'partB_response_val', 'partB_eliminations', 'predict_from_withstop', 'predict_from_onestepall', 'n_letter_all', 'n_sentence', 'n_token_verb', 'n_token_adj', 'n_token_noun', 'n_token_adv', 'n_token_content', 'n_token_content_unique', 'n_token_all', 'n_token_alpha', 'n_token_unique', 'word_diversity', 'mean_word_length', 'n_token_non_DaleChall', 'n_token_f

In [15]:
# Tokenize the data
model.resize_token_embeddings(len(tokenizer))


def tokenize_function(examples):
    return tokenizer(examples["parsed_xml_v1"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15644 [00:00<?, ? examples/s]

Map:   0%|          | 0/3911 [00:00<?, ? examples/s]

In [18]:
import sklearn
import numpy as np

In [19]:
# We'll use these weights later on to make up for the slightly imbalanced dataset
classes = np.unique(df["labels"])
class_weights = sklearn.utils.class_weight.compute_class_weight(
    "balanced", classes=classes, y=df["labels"]
)

class_weights = {clazz : weight for clazz, weight in zip(classes, class_weights)}

In [20]:
class_weights

{0: 0.9187221047686164, 1: 0.7314108318372232, 2: 1.8371852686959789}

In [21]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# AdamW Training
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=None,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.644500,0.573351
2,0.561400,0.561390
3,0.509700,0.576746


TrainOutput(global_step=1467, training_loss=0.5718707999324993, metrics={'train_runtime': 2360.9083, 'train_samples_per_second': 19.879, 'train_steps_per_second': 0.621, 'total_flos': 6217070824378368.0, 'train_loss': 0.5718707999324993, 'epoch': 3.0})

In [29]:
# Get predictions for parts graded via model
pred, actual, _ = trainer.predict(tokenized_datasets['test'])

In [32]:
actual

array([0, 0, 1, ..., 1, 0, 1])

In [33]:
# Convert probabilities to class labels
pred_labels = np.argmax(pred, axis=1)

# Calculate the quadratic weighted Cohen's kappa score
kappa = cohen_kappa_score(actual, pred_labels, weights='quadratic')

In [34]:
kappa

0.6964706160407158

In [22]:
from transformers import BertModel
import torch.nn as nn

class BertForSentenceClassification(BertModel):
    def __init__(self, config):
        super().__init__(config)
        
        ##### START YOUR CODE HERE #####
        # Add a linear classifier that map BERTs [CLS] token representation to the unnormalized
        # output probabilities for each class (logits).
        # Notes: 
        #  * See the documentation for torch.nn.Linear
        #  * You do not need to add a softmax, as this is included in the loss function
        #  * The size of BERTs token representation can be accessed at config.hidden_size
        #  * The number of output classes can be accessed at config.num_labels
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        ##### END YOUR CODE HERE #####
        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self, labels=None, **kwargs):
        outputs = super().forward(**kwargs)
        ##### START YOUR CODE HERE #####
        # Pass BERTs [CLS] token representation to this new classifier to produce the logits.
        # Notes:
        #  * The [CLS] token representation can be accessed at outputs.pooler_output
        cls_token_repr = outputs.pooler_output
        print(outputs.pooler_output.shape)
        logits = self.classifier(cls_token_repr)
        ##### END YOUR CODE HERE #####
        if labels is not None:
            outputs = (logits, self.loss(logits, labels))
        else:
            outputs = (logits,)
        return outputs

In [23]:
from torch.optim import AdamW
import time
import datetime
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import numpy as np
from sklearn.metrics import cohen_kappa_score


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def get_optimizer_and_scheduler(model, total_steps, lr=2e-5, weight_decay=0.01):
    # Apply weight decay to all parameters beside the biases or LayerNorm weights
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': weight_decay},
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        # Warmup learning rate for first 10% of training steps
        num_warmup_steps=int(0.10 * total_steps), 
        num_training_steps=total_steps,
    )
    return optimizer, scheduler

def train_model(model, epochs, train_dataloader, validation_dataloader,test_indexs):
    # Use GPU, if available
    device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    kappa_score_int = 0

    # Setup optimizer and LR scheduler 
    total_steps = len(train_dataloader) * epochs
    optimizer, scheduler = get_optimizer_and_scheduler(
        model, total_steps, lr=5e-5, weight_decay=0.01
    )

    loss_values = []
    eval_accs = []
        # Tracking variables
    

    for epoch in range(0, epochs):
        t0 = time.time()

        total_loss = 0
        model.train()
        predictions, true_labels = [], []

        with tqdm(train_dataloader, unit="batch") as train_pbar:
            for batch in train_pbar:
                train_pbar.set_description(f"Training (epoch {epoch + 1})")
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                model.zero_grad()        

                # Perform a forward pass (evaluate the model on this training batch).
                # This will return the loss because we have provided the `labels`.
                outputs = model(
                    input_ids=b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels
                )
                
                # The call to `model` always returns a tuple, so we need to pull the 
                # loss value out of the tuple.
                _, loss = outputs

                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value 
                # from the tensor.
                total_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                optimizer.step()

                # Update the learning rate.
                scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            
        
        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

        print("  * Average training loss: {0:.2f}".format(avg_train_loss))
        print("  * Training epoch took: {:}".format(format_time(time.time() - t0)))
            
        print("Running Validation...")

        t0 = time.time()
        model.eval()

        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            
            with torch.no_grad():        
                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                outputs = model(
                    input_ids=b_input_ids, 
                    attention_mask=b_input_mask
                )
            
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = outputs[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy
            # Track the number of batches
            nb_eval_steps += 1
            predictions.append(logits)
            true_labels.append(label_ids)
       
        predictions = np.concatenate(predictions)
        true_labels = np.concatenate(true_labels)

        predicted_labels = np.argmax(predictions, axis=1)
        kappa_score = cohen_kappa_score(true_labels, predicted_labels, labels=None, weights= 'quadratic', sample_weight=None)
        if kappa_score > kappa_score_int :
          kappa_score_int = kappa_score
          results_df = pd.DataFrame(index=test_indexs)
          results_df['indexes'] = test_indexs
          results_df['TrueValue'] = true_labels
          results_df['PredictedValue'] = predicted_labels

        

        avg_eval_acc = eval_accuracy/nb_eval_steps
        print("  * Accuracy: {0:.2f}".format(avg_eval_acc))
        print("  * Validation took: {:}".format(format_time(time.time() - t0)))
        print("Kappa Score = " + str(kappa_score))
        eval_accs.append(avg_eval_acc)
    print("Training complete!")
    return loss_values, eval_accs, results_df, kappa_score_int

In [24]:
def datapreprocess(df):
    MAX_LEN = 180
    bert_tokenizer = AutoTokenizer.from_pretrained('tbs17/MathBERT', do_lower_case=True)
    inputs = df.predict_from_withstop.values
    labels = df.assigned_score.values
    indexes = df.index.values
    inputs = ["[CLS] " + text + " [SEP]" for text in inputs]
    tokenized_inputs = bert_tokenizer(
        inputs,
        add_special_tokens=True,
        padding='max_length',
        max_length=MAX_LEN,
        return_tensors='pt',
        truncation=True
    )

    input_ids = tokenized_inputs['input_ids']
    attention_masks = tokenized_inputs['attention_mask']

    # Split the data into train and test sets
    train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, train_indexes, test_indexes = train_test_split(
        input_ids, attention_masks, labels, indexes, random_state=42, test_size=0.2, stratify=labels
    )

    return train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes


In [25]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

def train_valid_split(train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes, batch_size=32 ):
    # Use 80% for training and 20% for validation.

    train_labels = torch.tensor(train_labels)
    validation_labels = torch.tensor(test_labels)

    # Create the DataLoader for our training set.
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

    # Create the DataLoader for our validation set.
    validation_data = TensorDataset(test_inputs, test_masks, validation_labels)
    validation_dataloader = DataLoader(validation_data, shuffle=False, batch_size=batch_size)

    return train_dataloader, validation_dataloader

In [26]:
from transformers import BertModel
import torch.nn as nn

class BertCNNLSTMClassifier(BertModel):
    def __init__(self, config):
        super().__init__(config)
        
        self.lstm_hidden_size = 256
        self.lstm_num_layers = 4
        self.cnn_num_filters = 256
        self.cnn_kernel_size = 3
        
        # Add a linear classifier that maps the concatenated representation to the unnormalized
        # output probabilities for each class (logits).
        self.classifier = nn.Linear(config.hidden_size + self.lstm_hidden_size * 2 + self.cnn_num_filters, config.num_labels)
        
        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(config.hidden_size, self.lstm_hidden_size, num_layers=self.lstm_num_layers, batch_first=True, bidirectional=True)
        
        # CNN layer
        self.cnn = nn.Conv1d(
            in_channels=self.lstm_hidden_size * 2,
            out_channels=self.cnn_num_filters,
            kernel_size=self.cnn_kernel_size,
            padding=(self.cnn_kernel_size - 1) // 2
        )
        
        # Activation function
        self.activation = nn.ReLU()
        
        # Loss function
        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self, labels=None, **kwargs):
        outputs = super().forward(**kwargs)
        
        # Pass BERT's [CLS] token representation to the LSTM layer
        cls_token_repr = outputs.pooler_output
        lstm_output, _ = self.lstm(cls_token_repr.unsqueeze(1))
        
        # Reshape the LSTM output for CNN
        lstm_output = lstm_output.permute(0, 2, 1)  # Reshape for CNN
        
        # Pass the LSTM output through the CNN layer
        cnn_output = self.cnn(lstm_output)
        cnn_output = cnn_output.squeeze(2)  # Remove the last dimension
        
        # Reshape the LSTM output for concatenation
        lstm_output = lstm_output.squeeze(2)
        
        # Concatenate the [CLS] token representation, LSTM output, and CNN output
        concatenated_repr = torch.cat((cls_token_repr, lstm_output, cnn_output), dim=1)
        
        # Pass the concatenated representation to the linear classifier
        logits = self.classifier(concatenated_repr)
        
        if labels is not None:
            outputs = (logits, self.loss(logits, labels))
        else:
            outputs = (logits,)
        
        return outputs


In [27]:
model = BertCNNLSTMClassifier.from_pretrained(
    "distilbert-base-uncased",  # the name of the pretrained model
    num_labels=df['labels'].nunique(),  ) 

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertCNNLSTMClassifier: ['distilbert.transformer.layer.1.attention.q_lin.bias', 'distilbert.transformer.layer.2.attention.out_lin.weight', 'distilbert.embeddings.word_embeddings.weight', 'vocab_projector.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.2.attention.q_lin.bias', 'distilbert.transformer.layer.1.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.1.attention.out_lin.weight', 'distilbert.transformer.layer.2.output_layer_norm.bias', 'distilbert.transformer.layer.5.attention.out_lin.weight', 'vocab_transform.bias', 'distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.lay

In [28]:
train_inputs = tokenized_datasets['train']['input_ids']
train_masks = tokenized_datasets['train']['attention_mask'] 
train_labels = tokenized_datasets['train']['labels']
test_inputs = tokenized_datasets['test']['input_ids']
test_masks = tokenized_datasets['test']['attention_mask'] 
test_labels = tokenized_datasets['test']['labels']
test_indexes = tokenized_datasets['test']['student_id']

In [29]:
# Convert data to tensors
train_inputs = torch.tensor(train_inputs)
train_masks = torch.tensor(train_masks)
train_labels = torch.tensor(train_labels)
test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)
test_labels = torch.tensor(test_labels)

In [30]:
train_dataloader,validation_dataloader= train_valid_split(train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes )

<ipython-input-25-94c433efbdff>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
<ipython-input-25-94c433efbdff>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_labels = torch.tensor(test_labels)


In [ ]:
mathbert_loss_vals, mathbert_eval_accs, results_df, kappa_score = train_model(model=model,
    epochs=2,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader, test_indexs=test_indexes)

Training (epoch 1):   0%|          | 1/489 [01:45<14:16:00, 105.25s/batch]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in train_model:92                                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    489 │   │   )                                                                                 │
│    490                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:200 in backward               │
│                                                                                                  │
│   197 │   # The reason we repeat same the comment below is that                                  │
│   198 │   # some Python versions print out the first line of a multi-line function               │
│   199 │   # calls in the traceback and some print out the last line                              │
│ ❱ 200 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   201 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   202 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   203                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [38]:
# AdamW Training
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=None,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 19>:19                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1929 in _inner_training_loop     │
│                                                                                                  │
│   1926 │   │   │   │   │   with model.no_sync():                                                 │
│   1927 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1928 │   │   │   │   else:                                                                     │
│ ❱ 1929 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1930 │   │   │   │                                                                             │
│   1931 │   │   │   │   if (                                                                      │
│   1932 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2699 in training_step            │
│                                                                                                  │
│   2696 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2697 │   │                                                                                     │
│   2698 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2699 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2700 │   │                                                                                     │
│   2701 │   │   if self.args.n_gpu > 1:                                                           │
│   2702 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2731 in compute_loss             │
│                                                                                                  │
│   2728 │   │   │   labels = inputs.pop("labels")                                                 │
│   2729 │   │   else:                                                                             │
│   2730 │   │   │   labels = None                                                                 │
│ ❱ 2731 │   │   outputs = model(**inputs)                                                         │
│   2732 │   │   # Save past state if it exists              

In [27]:
def train(df, name) :
  train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes = datapreprocess(df)
  train_dataloader,validation_dataloader= train_valid_split(train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels, test_indexes )
  mathbert = BertCNNLSTMClassifier.from_pretrained(
    "tbs17/MathBERT",  # the name of the pretrained model
    num_labels=df['assigned_score'].nunique()+1,  ) 
  mathbert_loss_vals, mathbert_eval_accs, results_df, kappa_score = train_model(model=mathbert,
    epochs=6,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader, test_indexs=test_indexes)
  PATH1 = "/content/drive/MyDrive/NAEP_Comp/Bert+CNN/" + name  +".pt"
  #torch.save(mathbert, PATH1)
  results_df.to_csv('/content/drive/MyDrive/NAEP_Comp/MathBert+CNN/' + str(name) + '.csv', index=False)
  return kappa_score
   # the number of classes in our downstream task 



In [28]:
results = {}
for i, df in enumerate(dfs, start = 7):
    name = unique_accessions[i]
    df = dfs[name]
    print(name)
    df['predict_from_withstop'] = df['predict_from_withstop'].apply(extract_first_string)
    score = train(df, name)
    results[name] = [score]  # Store score as a list

    print(score)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('/content/drive/MyDrive/NAEP_Comp/MathBert+CNN/Results_Cohen_' + str(name) + '.csv', index=False)

VH304954


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:6                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'extract_first_string' is not defined

In [ ]:
inputs = df.predict_from.values
labels = df.assigned_score.values
print("Train data size ", len(inputs))
print('* Original:  ', inputs[0])
# Print the sentence split into tokens.
print('* Tokenized: ', bert_tokenizer.tokenize(inputs[0]))
# Print the sentence mapped to token ids.
print('* Token IDs: ', bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(inputs[0])))

In [ ]:
# Set the maximum sequence length.
MAX_LEN = 100

# Print BERTs special PAD token and its index in the vocabulary
print(f'Padding token: "{bert_tokenizer.pad_token}", ID: {bert_tokenizer.pad_token_id}')

In [ ]:
tokenized_inputs = bert_tokenizer(
    inputs.tolist(),          # Input text
    add_special_tokens=True,  # add '[CLS]' and '[SEP]'
    padding='max_length',     # pad to a length specified by the max_length
    max_length=MAX_LEN,       # truncate all sentences longer than max_length
    return_tensors='pt',  
    truncation = True    # return everything we need as PyTorch tensors
)

input_ids = tokenized_inputs['input_ids']
attention_masks = tokenized_inputs['attention_mask']

# Print sentence 0, now as a list of IDs.
print('Original: ', tokenized_inputs['input_ids'][0])
print('* Token IDs:', tokenized_inputs['attention_mask'][0])
print('* Tokenized:', bert_tokenizer.decode(tokenized_inputs['input_ids'][0]))
print('* Attention_mask', tokenized_inputs['attention_mask'][0])

In [ ]:
bert_train_dataloader, bert_validation_dataloader = train_valid_split(
    input_ids=input_ids,
    attention_masks=attention_masks,
    labels=labels,
    batch_size=32
)

In [ ]:
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining


In [ ]:
mathbert = BertForSentenceClassification.from_pretrained(
    "tbs17/MathBERT",  # the name of the pretrained model
    num_labels=4,      # the number of classes in our downstream task 
)

In [ ]:
# Model parameters visualization
params = list(mathbert.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer Layer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# About 2-3 seconds per epoch using GPU
mathbert_loss_vals, mathbert_eval_accs = train_model(
    model=mathbert,
    epochs=1,
    train_dataloader=bert_train_dataloader,
    validation_dataloader=bert_validation_dataloader
)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

def plot_loss_and_acc(loss_vals, eval_accs):
    sns.set(style='darkgrid')
    sns.set(font_scale=1.5)
    plt.rcParams["figure.figsize"] = (12,6)
    fig, ax1 = plt.subplots(1,1)
    ax1.plot(loss_vals, 'b-o', label = 'training loss')
    ax2 = ax1.twinx()
    ax2.plot(eval_accs, 'y-o', label = 'validation accuracy')
    ax2.set_title("Training loss and validation accuracy")
    ax2.set_xlabel("Epoch")
    ax1.set_ylabel("Loss", color='b')
    ax2.set_ylabel("Accuracy", color='y')
    ax1.tick_params(axis='y', rotation=0, labelcolor='b' )
    ax2.tick_params(axis='y', rotation=0, labelcolor='y' )
    plt.show()

In [ ]:
plot_loss_and_acc(mathbert_loss_vals, mathbert_eval_accs)

In [ ]:
mathbert_frozen = BertForSentenceClassification.from_pretrained(
    "tbs17/MathBERT",  # the name of the pretrained model
    num_labels=4,      # the number of classes in our downstream task
)

In [ ]:
for name, param in mathbert_frozen.named_parameters():
	# Only compute gradients for parameters of our
	# newly added classifier. BERT will not be trained.
	if 'classifier' not in name:
		param.requires_grad = False

In [ ]:
# About 1 second per epoch on GPU
mathbert_frozen_loss_vals, mathbert_frozen_eval_accs = train_model(
    model=mathbert_frozen,
    epochs=3, 
    train_dataloader=bert_train_dataloader,
    validation_dataloader=bert_validation_dataloader
)